### Imports

In [ ]:
ptrain_url = 'https://raw.githubusercontent.com/sidt-ai/MH-hackathons/main/dare_in_reality_2021/data/processed/proc_train.csv'
ptest_url = 'https://raw.githubusercontent.com/sidt-ai/MH-hackathons/main/dare_in_reality_2021/data/processed/proc_test.csv'
submission_url = 'https://raw.githubusercontent.com/sidt-ai/MH-hackathons/main/dare_in_reality_2021/data/raw/submission.csv'

In [ ]:
!pip install --quiet optuna

     |████████████████████████████████| 308 kB 5.4 MB/s 
     |████████████████████████████████| 80 kB 6.2 MB/s 
     |████████████████████████████████| 209 kB 28.4 MB/s 
     |████████████████████████████████| 75 kB 2.6 MB/s 
     |████████████████████████████████| 49 kB 4.3 MB/s 
     |████████████████████████████████| 112 kB 27.9 MB/s 
     |████████████████████████████████| 149 kB 29.0 MB/s 


In [ ]:
!pip install --quiet catboost

     |████████████████████████████████| 76.3 MB 36 kB/s 


In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from catboost import CatBoostRegressor

import optuna
from optuna.samplers import TPESampler

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

N_SPLITS = 5
SEED = 2311

# Model 1 - CatBoost, no weather data [Public LB - ]

In [ ]:
ptrain = pd.read_csv(ptrain_url)
ptest = pd.read_csv(ptest_url)

### Creating Folds for Cross-validation

In [ ]:
ptrain['fold'] = -1

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

for fold, (_, val_idx) in enumerate(kf.split(X=ptrain)):
  ptrain.loc[val_idx, 'fold'] = fold

In [ ]:
ptrain.shape[0], ptest.shape[0]

(10054, 420)

In [ ]:
ptrain.columns

Index(['CAR_NUMBER', 'LAP_NUMBER', 'LAP_TIME', 'LAP_IMPROVEMENT',
       'CROSSING_FINISH_LINE_IN_PIT', 'S1_IMPROVEMENT', 'S2_IMPROVEMENT',
       'S3_IMPROVEMENT', 'KPH', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE',
       'DRIVER_NAME', 'PIT_TIME', 'TEAM', 'LOCATION', 'EVENT', 'PITSTOP_TAKEN',
       'EVENT_TYPE', 'fold'],
      dtype='object')

In [ ]:
features = [f for f in ptrain.columns if f not in ('LAP_TIME', 'fold')]
cat_features = [f for f in features if f not in ('KPH', 'PIT_TIME', 'S1_LARGE', 'S2_LARGE', 'S3_LARGE')]

### Hyperparameter tuning using Optuna

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(ptrain[features], ptrain.LAP_TIME,
                                              test_size=0.2, random_state=SEED)

In [ ]:
base_params = {
    'iterations': 5000,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'score_function': 'L2',
    'task_type': 'GPU',
    'cat_features': cat_features,
    'use_best_model': True,
    'random_seed': SEED
}

In [ ]:
def objective(trial, xtrain, ytrain, xval, yval, base_params):

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_uniform('l2_leaf_reg', 0.1, 10),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', 
                                                    ['Bayesian', 'Poisson', 'No']),
        'grow_policy': trial.suggest_categorical('grow_policy', 
                                                 ['SymmetricTree', 'Lossguide'])
    }
    
    model = CatBoostRegressor(**base_params, **param_grid)
    
    model.fit(
        xtrain, ytrain,
        eval_set=(xval, yval),
        early_stopping_rounds=100,
        verbose_eval=100
    )
    
    predictions = model.predict(xval)
    return mean_squared_log_error(yval, predictions, squared=False)

In [ ]:
%%time
study = optuna.create_study(sampler=TPESampler(seed=SEED), 
                            direction='minimize', 
                            study_name='dir2021_catboost')

study.optimize(
    lambda trial: objective(trial, xtrain, ytrain, xval, yval, base_params), 
    n_trials=50
)

[I 2021-11-22 12:47:01,579] A new study created in memory with name: dir2021_catboost


0:	learn: 23.3019442	test: 23.9740440	best: 23.9740440 (0)	total: 43.7ms	remaining: 3m 38s
100:	learn: 21.9519980	test: 23.2765798	best: 23.2260595 (64)	total: 3.25s	remaining: 2m 37s
bestTest = 23.22605947
bestIteration = 64
Shrink model to first 65 iterations.


[I 2021-11-22 12:47:09,288] Trial 0 finished with value: 0.21793188570186026 and parameters: {'learning_rate': 0.03894082102082652, 'depth': 9, 'l2_leaf_reg': 2.2658391209721263, 'bootstrap_type': 'No', 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.21793188570186026.


0:	learn: 23.2346375	test: 23.9138535	best: 23.9138535 (0)	total: 69.8ms	remaining: 5m 48s
100:	learn: 21.4161533	test: 23.2762073	best: 23.2197133 (30)	total: 5.55s	remaining: 4m 29s
bestTest = 23.21971327
bestIteration = 30
Shrink model to first 31 iterations.


[I 2021-11-22 12:47:18,544] Trial 1 finished with value: 0.2177702984992022 and parameters: {'learning_rate': 0.07699736160829906, 'depth': 7, 'l2_leaf_reg': 0.6319937645727293, 'bootstrap_type': 'No', 'grow_policy': 'SymmetricTree'}. Best is trial 1 with value: 0.2177702984992022.


...

0:	learn: 23.2245082	test: 23.9020877	best: 23.9020877 (0)	total: 26.4ms	remaining: 2m 12s
100:	learn: 22.1937878	test: 23.2517891	best: 23.2091597 (28)	total: 2.71s	remaining: 2m 11s
bestTest = 23.20915966
bestIteration = 28
Shrink model to first 29 iterations.


[I 2021-11-22 12:52:27,418] Trial 48 finished with value: 0.21776376801234051 and parameters: {'learning_rate': 0.08545500178407597, 'depth': 4, 'l2_leaf_reg': 2.555316431667239, 'bootstrap_type': 'No', 'grow_policy': 'SymmetricTree'}. Best is trial 43 with value: 0.21763347031045138.


0:	learn: 23.2031715	test: 23.8808709	best: 23.8808709 (0)	total: 20.1ms	remaining: 1m 40s
100:	learn: 22.2660869	test: 23.2437399	best: 23.2040116 (22)	total: 2.14s	remaining: 1m 43s
bestTest = 23.20401164
bestIteration = 22
Shrink model to first 23 iterations.


[I 2021-11-22 12:52:31,637] Trial 49 finished with value: 0.21764731152847647 and parameters: {'learning_rate': 0.09964303073248612, 'depth': 3, 'l2_leaf_reg': 1.920333567788793, 'bootstrap_type': 'No', 'grow_policy': 'SymmetricTree'}. Best is trial 43 with value: 0.21763347031045138.


CPU times: user 4min 53s, sys: 2min 38s, total: 7min 31s
Wall time: 5min 30s


In [ ]:
best_params = study.best_params
print('Best params:')
for key, value in best_params.items():
    print(f'\t{key}: {value}')

Best params:
	learning_rate: 0.09654703295900584
	depth: 3
	l2_leaf_reg: 3.392212884308976
	bootstrap_type: No
	grow_policy: SymmetricTree


In [ ]:
del xtrain, xval, ytrain, yval

In [ ]:
def custom_cross_val_predict(train, test, features):
    oof_preds = {}
    test_preds = []
    scores = []
    
    cv_start = time.time()
    
    for fold in range(N_SPLITS):
        print('-' * 40)
        
        xtrain = train[train.fold != fold].reset_index(drop=True)
        ytrain = xtrain.LAP_TIME

        xval = train[train.fold == fold].reset_index(drop=True)
        yval = xval.LAP_TIME    
        val_idx = xval.index.values.tolist()
        
        fold_start = time.time()
        
        model = CatBoostRegressor(**base_params, **best_params)

        model.fit(
            xtrain[features], ytrain,
            eval_set=(xval[features], yval),
            early_stopping_rounds=100,
            verbose_eval=100
        )
        val_preds = model.predict(xval[features])
        oof_preds.update(dict(zip(val_idx, val_preds)))
        auc = mean_squared_log_error(xval.LAP_TIME, val_preds, squared=False)
        scores.append(auc)
        
        fold_end = time.time()
        
        print(f'Fold #{fold}: AUC = {auc:.5f}\t[Time: {fold_end - fold_start:.2f} secs]')
        
        test_preds.append(model.predict(test[features]))

        del xtrain, ytrain, xval, yval
        
    cv_end = time.time()
    print(f'\nAverage RMSLE = {np.mean(scores):.5f} with std. dev. = {np.std(scores):.5f}')
    print(f'[Total time: {cv_end - cv_start:.2f} secs]\n')
    
    oof_preds = pd.DataFrame.from_dict(oof_preds, orient='index').reset_index()
    test_preds = np.mean(np.column_stack(test_preds), axis=1)
    
    return oof_preds, test_preds

In [ ]:
oof_preds, test_preds = custom_cross_val_predict(ptrain, ptest, features)

----------------------------------------
0:	learn: 23.2077632	test: 23.8845146	best: 23.8845146 (0)	total: 17.8ms	remaining: 1m 29s
100:	learn: 22.2769045	test: 23.2190266	best: 23.2029883 (23)	total: 2.05s	remaining: 1m 39s
bestTest = 23.20298833
bestIteration = 23
Shrink model to first 24 iterations.
Fold #0: AUC = 0.21771	[Time: 4.09 secs]
----------------------------------------
0:	learn: 23.2253661	test: 23.8331287	best: 23.8331287 (0)	total: 18.9ms	remaining: 1m 34s
100:	learn: 22.3635292	test: 23.1037619	best: 23.0869556 (33)	total: 2.01s	remaining: 1m 37s
bestTest = 23.08695565
bestIteration = 33
Shrink model to first 34 iterations.
Fold #1: AUC = 0.21466	[Time: 4.18 secs]
----------------------------------------
0:	learn: 23.3788007	test: 23.2006723	best: 23.2006723 (0)	total: 19.1ms	remaining: 1m 35s
100:	learn: 22.4799958	test: 22.5208975	best: 22.5187563 (98)	total: 2.09s	remaining: 1m 41s
bestTest = 22.51875634
bestIteration = 98
Shrink model to first 99 iterations.
Fold #

In [ ]:
submission = pd.read_csv(submission_url)
submission.LAP_TIME = test_preds
submission.to_csv('sub2_catboost_newproc_noweather.csv', index=False)

In [ ]:
!head sub2_catboost_newproc_noweather.csv

LAP_TIME
98.14868657813058
97.68700761440559
97.06044196338917
97.80922575878795
97.54292190513225
96.89852526722242
97.9170745583775
97.74799216990941
95.94249607191304


# Model 2